In [1]:
import numpy as np
from openfermion.chem import geometry_from_pubchem
from quri_parts.chem import cas
from quri_parts.pyscf.mol import get_spin_mo_integrals_from_mole
from quri_parts.openfermion.mol import get_qubit_mapped_hamiltonian
from quri_parts.openfermion.ansatz import TrotterUCCSD
from quri_parts.core.operator import get_sparse_matrix
from pyscf import gto, scf, mcscf, cc
from quri_parts_qsci import qsci
from quri_parts.chem.mol import ActiveSpaceMolecularOrbitals, ActiveSpace
from quri_parts.pyscf.mol import PySCFMolecularOrbitals
from quri_parts.core.state import quantum_state

from typing import Sequence

import numpy.typing as npt

# Generate Hamiltonian of AB (azobenzene)
1. Hamiltonian
2. Exact ground state energy
3. Approximated energy includes electron correlation effects from single and double excitations. 

In [2]:
def from_Hartree_to_eV(Hartree_energy):
    return Hartree_energy/27.21

In [5]:
atom = geometry_from_pubchem("azobenzene")
mole = gto.M(atom=atom)
mf = scf.RHF(mole).run()

active_space = cas(4, 4)
hamiltonian, mapping = get_qubit_mapped_hamiltonian(
    *get_spin_mo_integrals_from_mole(mole, mf.mo_coeff, active_space)
)

vals, vecs = np.linalg.eigh(get_sparse_matrix(hamiltonian).toarray())

casci_gs_e = np.min(vals)
ccsd = cc.CCSD(mf).run(verbose=0)

print("Hamiltonian: ", hamiltonian)
print(f"The exact ground state energy is: {from_Hartree_to_eV(casci_gs_e)} eV")
print(f"The coupled cluster singles and doubles energy is: {from_Hartree_to_eV(ccsd.e_tot)} eV")

converged SCF energy = -562.053344724961
Hamiltonian:  (-561.2163615469058+0j)*I + (0.07929207448091026+0j)*Z0 + (0.07929207448091027+0j)*Z1 + (0.041737792195638924+0j)*Z2 + (0.04173779219563891+0j)*Z3 + (-0.1111972682302581+0j)*Z4 + (-0.11119726823025808+0j)*Z5 + (-0.08721715886291209+0j)*Z6 + (-0.08721715886291209+0j)*Z7 + (0.05488979713052594+0j)*Z0 Z1 + -0.0015964182706902658*X0 Z1 X2 + -0.0015964182706902658*Y0 Z1 Y2 + (0.02999048482644298+0j)*Z0 Z2 + (0.053544511923058855+0j)*Z0 Z3 + -1.9995608177153797e-05*X0 Z1 Z2 Z3 X4 + -1.9995608177153797e-05*Y0 Z1 Z2 Z3 Y4 + (0.049962972036537816+0j)*Z0 Z4 + (0.051003560119276446+0j)*Z0 Z5 + 0.014257630971906847*X0 Z1 Z2 Z3 Z4 Z5 X6 + 0.014257630971906847*Y0 Z1 Z2 Z3 Z4 Z5 Y6 + (0.0429566535937987+0j)*Z0 Z6 + (0.05489765632566161+0j)*Z0 Z7 + (0.053544511923058855+0j)*Z1 Z2 + -0.0015964182706902658*X1 Z2 X3 + -0.0015964182706902658*Y1 Z2 Y3 + (0.02999048482644298+0j)*Z1 Z3 + (0.051003560119276446+0j)*Z1 Z4 + -1.9995608177153797e-05*X1 Z2 Z3 

In [6]:
[from_Hartree_to_eV(val-vals[0]) for val in vals[:10]]

[0.0,
 0.00831725405732661,
 0.008317254057330787,
 0.008373638547204007,
 0.008373638547224898,
 0.008373638547229076,
 0.00900472860596129,
 0.009572315526184831,
 0.009572315526193188,
 0.009583214165120814]

In [48]:
from typing import Sequence

import numpy.typing as npt


def ccsd_param_to_circuit_param(
    uccsd: TrotterUCCSD,
    n_electrons: int,
    t1: npt.NDArray[np.complex128],
    t2: npt.NDArray[np.complex128],
) -> Sequence[float]:
    in_param_list = uccsd.param_mapping.in_params
    param_list = []

    for param in in_param_list:
        name_split = param.name.split("_")
        if name_split[0] == "s":
            _, i_str, j_str = name_split
            i, j = int(i_str), int(j_str) - n_electrons // 2
            param_list.append(t1[i, j])

        if name_split[0] == "d":
            _, i_str, j_str, a_str, b_str = name_split
            i, j, b, a = (
                int(i_str),
                int(j_str),
                int(b_str) - n_electrons // 2,
                int(a_str) - n_electrons // 2,
            )
            param_list.append(t2[i, j, a, b])
    return param_list


In [46]:
from quri_parts.qulacs.estimator import (
    create_qulacs_vector_concurrent_parametric_estimator,
)
from quri_parts.core.state import quantum_state, apply_circuit

TROTTER_STEPS = 1
USE_SINGLES = True
REDUCE_PARAMETER = True
BASIS_STATES = 64

uccsd = TrotterUCCSD(
        active_space.n_active_orb * 2,
        active_space.n_active_ele,
        trotter_number=TROTTER_STEPS,
        use_singles=USE_SINGLES,
        singlet_excitation=REDUCE_PARAMETER,
    )
param = ccsd_param_to_circuit_param(uccsd, active_space.n_active_ele, ccsd.t1, ccsd.t2)
hf_state = quantum_state(active_space.n_active_orb * 2, bits=2**active_space.n_active_ele - 1)
state = apply_circuit(uccsd, hf_state)
bound_state = state.bind_parameters(param)

In [ ]:
estimator = create_qulacs_vector_concurrent_parametric_estimator()
estimate = estimator(hamiltonian, state, [param])
print("The coupled cluster singles and doubles energy is:", from_Hartree_to_eV(ccsd.e_tot))
print(
    "The Trotter unitary coupled cluster singles and doubles energy is:",
    from_Hartree_to_eV(estimate[0].value.real),
)

The coupled cluster singles and doubles energy is: -20.69185732651236
The Trotter unitary coupled cluster singles and doubles energy is: -20.65613213482361


## Excited states

In [34]:
exc_bits = (2**active_space.n_active_ele - 1) ^ 0b00010100 
excited_state = quantum_state(active_space.n_active_orb * 2, bits=exc_bits) 
ex_state_circuit = apply_circuit(uccsd, excited_state)
bound1_state = ex_state_circuit.bind_parameters(param)


In [38]:
estimator = create_qulacs_vector_concurrent_parametric_estimator()
estimate = estimator(hamiltonian, ex_state_circuit, [param])
print("The coupled cluster singles and doubles energy is:", from_Hartree_to_eV(ccsd.e_tot))
print(
    "The Trotter unitary coupled cluster singles and doubles energy is:",
    from_Hartree_to_eV(estimate[0].value.real),
)

The coupled cluster singles and doubles energy is: -20.656290717085778
The Trotter unitary coupled cluster singles and doubles energy is: -20.647456430789756


# QSCI

In [11]:
from quri_vm import VM

from quri_parts.backend.devices import star_device
from quri_parts.backend.units import TimeUnit, TimeValue

from typing import Iterable

from quri_parts.circuit import ImmutableQuantumCircuit
from quri_parts.core.sampling import ConcurrentSampler, MeasurementCounts

BASIS_STATES = 4
TOTAL_SHOTS = 50000

star_vm = VM.from_device_prop(
    star_device.generate_device_property(
        qubit_count=4,
        code_distance=7,
        qec_cycle=TimeValue(1, TimeUnit.MICROSECOND)
    )
)

def create_concurrent_sampler_from_vm(
    vm: VM,
) -> ConcurrentSampler:
    """Create a simple :class:`~ConcurrentSampler` using a
    :class:`~SamplingBackend`."""

    def sampler(
        shot_circuit_pairs: Iterable[tuple[ImmutableQuantumCircuit, int]]
    ) -> Iterable[MeasurementCounts]:
        jobs = [
            vm.sample(circuit, n_shots) for circuit, n_shots in shot_circuit_pairs
        ]
        return map(lambda j: j, jobs)

    return sampler

In [12]:
from quri_parts_qsci import qsci

sampler = create_concurrent_sampler_from_vm(star_vm)
bound_state = state.bind_parameters(param)
eigs, _ = qsci(
    hamiltonian, [bound_state], sampler, total_shots=TOTAL_SHOTS, num_states_pick_out=4
)

In [16]:
err = lambda x: abs(x / casci_gs_e - 1)
print(f"The exact GS energy is {from_Hartree_to_eV(casci_gs_e)}")
print(
    f"The coupled cluster energy is {from_Hartree_to_eV(ccsd.e_tot)}, with relative error {err(ccsd.e_tot)}"
)
print(
    f"The Trotter UCCSD energy is {from_Hartree_to_eV(estimate[0].value.real)}, with relative error {err(estimate[0].value.real)}"
)
print(f"The qsci gs energy is {from_Hartree_to_eV(eigs[0])}, with relative error {err(eigs[0])}")

The exact GS energy is -20.656290746782773
The coupled cluster energy is -20.656290717085778, with relative error 1.4376732027088224e-09
The Trotter UCCSD energy is -20.65629072472936, with relative error 1.067636645757375e-09
The qsci gs energy is -20.656288686567446, with relative error 9.973791281758793e-08
